# This notebook is intended to run a series of tests to ensure the system responds as expected
## It begins with LATISS, but can also be used with the telescope components only

In [ ]:
import sys
import asyncio
import time
import os

import numpy as dr

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

from lsst.ts.observatory.control.utils import RotType

In [ ]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
#get classes and start them
domain = salobj.Domain()

In [ ]:
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
with_latiss=True

In [ ]:
# Enable LATISS
if with_latiss:
    await latiss.enable()

In [ ]:
# Take bias and make sure it lands
if with_latiss:
    latiss.rem.atarchiver.evt_imageInOODS.flush()
    await latiss.take_bias(nbias=1)
    tmp = await latiss.rem.atarchiver.evt_imageInOODS.next(flush=False, timeout=10)
    print(tmp)
    if tmp.statusCode != 0:
        raise AssertionError('Ingestion was not successful!')

In [ ]:
# Check that the bias pops up in the monitor (https://roundtable.lsst.codes/rubintv/monitor_current)
# If it fails, let Patrick or Merlin know

In [ ]:
# Take an engineering test frame (analogous to a science frame) and make sure it lands
if with_latiss:
    latiss.rem.atarchiver.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter='FELH0600',grating=3)
    tmp = await latiss.rem.atarchiver.evt_imageInOODS.next(flush=False, timeout=10)
    print(tmp)
    if tmp.statusCode != 0:
        raise AssertionError('Ingestion was not successful!')

In [ ]:
# Enable ATCS
await atcs.enable()

In [ ]:
# Make sure valves are open
await atcs.open_valves()
pressure = await atcs.rem.atpneumatics.tel_mainAirSourcePressure.next(flush=True, timeout=5)
if pressure.pressure > 300000:
    print(f'Air pressure is {pressure.pressure:0.0f} Pascals, which is fine.')
else:
    print(f'Air pressure is {pressure.pressure}, which is too low. It needs to be between ~275790 and ~413000 Pascals (40 and 60 PSI). Check that compressor and dryer is running. Then check that the regulator inside the pier is set correctly. ')

In [ ]:
# Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above)
# Note there is a race condition in the ATPneumatics that might result in this failing, so you'll have to run in twice
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

In [ ]:
pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.aget(timeout=5)
print(pressure)

In [ ]:
# Turn off ATAOS correction(s), without spectrograph (if the tests were skipped above)
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

In [ ]:
# put mirror back on hardpoints
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)

In [ ]:
# Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above)
# This should not ever fail.
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

In [ ]:
# Turn off dome following
await atcs.disable_dome_following()

In [ ]:
# Test Point AzEl
start_az=10
start_el=75
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

In [ ]:
# Stop tracking
tmp=await atcs.stop_tracking()
print(tmp)

In [ ]:
# Now start tracking a siderial target, but starting from the same position
coord=atcs.radec_from_azel(az=start_az+2, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False)

In [ ]:
# Stop tracking
await atcs.stop_tracking()

In [ ]:
# Check that dome moves
dome_az = await atcs.rem.atdome.tel_position.next(flush=True,timeout=10)
print(f'Dome currently thinks it is at an azimuth position of {dome_az.azimuthPosition}.\n Note the dome may not be properly homed at this time')
d_az=15
await atcs.rem.atdome.cmd_moveAzimuth.set_start(azimuth=dome_az.azimuthPosition+d_az)

In [ ]:
# Turn off ATAOS correction(s)
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)
# put mirror back on the hardpoints
pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.aget(timeout=5)
print(pressure)

In [ ]:
# shut off the valves so we can test the pneumatics
await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.start()
await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.start()

In [ ]:
# Open mirror covers and vents
# Note that there is currently a race conditions that might make this fail in the first attempt
await atcs.open_m1_cover()
await atcs.open_m1_vent()

In [ ]:
# Close mirror covers and vents
await atcs.close_m1_cover()
await atcs.close_m1_vent()

In [ ]:
# Setup to run flats - this will also home the dome
await atcs.prepare_for_flatfield()

In [ ]:
# Now close the mirror cover just to keep things safe - this will move the telescope to a higher position (El~70)
await atcs.close_m1_cover()
await atcs.close_m1_vent()

In [ ]:
# Turn on dome following which will align the dome with the telescope
await atcs.enable_dome_following()

In [ ]:
# Now do a full slew of the observatory

In [ ]:
# Turn on ATAOS correction(s), without spectrograph (if the tests were skipped above)
# Note there is a race condition in the ATPneumatics that might result in this failing, so you'll have to run in twice
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=with_latiss)

In [ ]:
# Now start tracking a siderial target, but starting from the same position
start_az=205-30
start_el=70-10
start_rot=0
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False)

In [ ]:
# Take an engineering test frame (analogous to a science frame) and make sure it lands
if with_latiss:
    latiss.rem.atarchiver.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter=3, grating=3)
    tmp = await latiss.rem.atarchiver.evt_imageInOODS.next(flush=False, timeout=10)
    print(tmp)
    if tmp.statusCode != 0:
        raise AssertionError('Ingestion was not successful!')

In [ ]:
# Let it track for at least a couple minutes

In [ ]:
# Try a new target
# Now start tracking a siderial target, but starting from the same position
start_az=205-15
start_el=70-5
start_rot=0
coord=atcs.radec_from_azel(az=start_az, el=start_el)
await atcs.slew_icrs(coord.ra, coord.dec, rot=start_rot, stop_before_slew=False)

In [ ]:
# Take an engineering test frame (analogous to a science frame) and make sure it lands
if with_latiss:
    latiss.rem.atarchiver.evt_imageInOODS.flush()
    await latiss.take_engtest(2, filter=0, grating=0)
    tmp = await latiss.rem.atarchiver.evt_imageInOODS.next(flush=False, timeout=10)
    print(tmp)
    if tmp.statusCode != 0:
        raise AssertionError('Ingestion was not successful!')

In [ ]:
# now shut everything down
await atcs.shutdown()

In [ ]:
await latiss.standby()